## 读取模型并计算SHAP values

In [ ]:
import sklearn

print(sklearn.__version__)

In [ ]:
import pickle
from shap import Explanation
import pandas as pd
import shap
from shap import LinearExplainer, KernelExplainer, Explanation
shap.initjs()
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import matplotlib.pyplot as plt

with open('metrics/best_model_item.pkl', 'rb') as f:
    best_model_item = pickle.load(f)
clf_stacking = best_model_item['clf']
best_model_index = best_model_item['index']
print("The index (random state) used for the best model is: ", best_model_index)

# Load the data from Excel
xls = pd.ExcelFile('split_csv/Stacking_indices39.xlsx')
df_train = pd.read_excel(xls, 'Hold Train')
df_test = pd.read_excel(xls, 'Hold Test')

# Extract features and labels
X_train = df_train.drop(columns=['Y', 'Unnamed: 0'])  
y_train = df_train['Y']

X_test = df_test.drop(columns=['Y', 'Unnamed: 0']) 
y_test = df_test['Y']

# 修改字段名称
new_column_names = {
    'RM-750_acc': 'accuracy in 750 ms',
    'Switch_cost': 'Switching cost',
    'IUF': 'Inhibition and Updating Factor',
    'Stroop_neutral_rt': 'reaction times in neutral stimuli',
    'Go_acc': 'accuracy in Go trials',
    'US': 'Updating and Switching Factor',
    'CBT_acc': 'accuracy in Corsi block test',
    'Stroop_interference effect_rt': 'stroop interference effect',
    'EFF': 'Executive Function Factor',
    'Stroop_congruent_rt': 'reaction times in congruent stimuli',
    'DSBT_Span': 'span in Digit span backward task',
    'CBT_Span': 'span in Corsi block test',
    'Nogo_acc': 'accuracy in Nogo trials',
    'Stroop_incongruent_rt': 'reaction times in incongruent stimuli',
    'IF': 'Inhibition Factor',
    'RM-1,750_acc': 'accuracy in 1,750 ms',
    'Go_rt': 'reaction times in Go trials',
    'Mixing_cost': 'Mixing cost',
    'ISF': 'Inhibition and Switching Factor'
}

X_train = X_train.rename(columns=new_column_names)
X_test = X_test.rename(columns=new_column_names)

# 交换标签0和1
y_train = 1 - y_train
y_test = 1 - y_test

with open('metrics/best_model_item.pkl', 'rb') as f:
    best_model_item = pickle.load(f)
clf_stacking = best_model_item['clf']

clf_stacking.fit(X_train, y_train)

np.random.seed(42) #使用的是 Monte Carlo 采样来估计这些值，存在一定随机性

explainer = shap.KernelExplainer(clf_stacking.predict, X_test) 
sv = explainer.shap_values(X_test)

data_dict = {
    'X_train': X_train,
    'X_test': X_test,
    'y_train': y_train,
    'y_test': y_test,
    'clf_stacking': clf_stacking,
    'shap_values': sv,
    'expected_value': explainer.expected_value,
    'feature_names': X_test.columns
}

with open('./data/shap_all_info.pkl', 'wb') as f: #存储为pkl，防止每次都得计算
    pickle.dump(data_dict, f)

In [ ]:
# 打印修改后的特征名称
print("Modified feature names in X_train:")
print(X_train.columns)

print("Modified feature names in X_test:")
print(X_test.columns)

In [ ]:
import pickle
from shap import Explanation
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
import shap
from shap import LinearExplainer, KernelExplainer, Explanation
shap.initjs()
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import matplotlib.pyplot as plt

with open('./data/shap_all_info.pkl', 'rb') as f:
    data_dict = pickle.load(f)
    
    
    
# Load the data from Excel
xls = pd.ExcelFile('split_csv/Stacking_indices39.xlsx')
df_train = pd.read_excel(xls, 'Hold Train')
df_test = pd.read_excel(xls, 'Hold Test')

# Extract features and labels
X_train = df_train.drop(columns=['Y', 'Unnamed: 0'])  
y_train = df_train['Y']

X_test = df_test.drop(columns=['Y', 'Unnamed: 0']) 
y_test = df_test['Y']


with open('metrics/best_model_item.pkl', 'rb') as f:
    best_model_item = pickle.load(f)
clf_stacking = best_model_item['clf']


clf_stacking.fit(X_train, y_train)


explainer = shap.KernelExplainer(clf_stacking.predict, X_test)

X_train = data_dict['X_train']
X_test = data_dict['X_test']
y_train = data_dict['y_train']
y_test = data_dict['y_test']
clf_stacking = data_dict['clf_stacking']
shap_values = data_dict['shap_values']
feature_names = data_dict['feature_names']
expected_value = data_dict['expected_value']

In [ ]:
import matplotlib.pyplot as plt

# Set the global font to be bold, size 12
plt.rcParams['font.weight'] = 'bold'

### 输出SHAP values


In [ ]:
# Predict the labels of the test set
y_pred = clf_stacking.predict(X_test)

# Create a dataframe for the comparison
prediction_comparison = pd.DataFrame({'y_true': y_test, 'y_pred': y_pred}) # Notice the change to 'y_test'

# Only keep the correctly predicted rows
correct_predictions = prediction_comparison[prediction_comparison['y_true'] == prediction_comparison['y_pred']]

# Add the correctly predicted flag to the shap_values dataframe
shap_values_df = pd.DataFrame(shap_values, columns=X_test.columns) # Notice the change to 'shap_values'
shap_values_df['Correct Prediction'] = shap_values_df.index.isin(correct_predictions.index)

# Add the predicted class to the shap_values dataframe
shap_values_df['Predicted Class'] = y_pred

# Create a new DataFrame to store the original feature values
original_values_df = X_test.copy()
original_values_df['Correct Prediction'] = original_values_df.index.isin(correct_predictions.index)
original_values_df['Predicted Class'] = y_pred

# Use pandas' ExcelWriter function to write the two DataFrames into different worksheets in the same Excel file
with pd.ExcelWriter('./table/output_values-测试集所有样本.xlsx') as writer:
    shap_values_df.to_excel(writer, sheet_name='SHAP Values')
    original_values_df.to_excel(writer, sheet_name='Original Feature Values')

In [ ]:
xls = pd.ExcelFile('split_csv/Stacking_indices39.xlsx')

# 加载 'Test Data' 数据表
df_test_data = pd.read_excel(xls, 'Test Data')

# 添加到 'SHAP Values' 数据表
shap_values_df = pd.concat([shap_values_df, df_test_data], axis=1)


# 删除 'Predicted Class' 为 1 的行
shap_values_df = shap_values_df[shap_values_df['Predicted Class'] != 0]

# 将更新后的数据表存储到一个新的 Excel 文件中
with pd.ExcelWriter('./table/output_values_filtered-被预测为患者的样本.xlsx') as writer:
    shap_values_df.to_excel(writer, sheet_name='SHAP Values Filtered')
    original_values_df.to_excel(writer, sheet_name='Original Feature Values')

In [ ]:
xls = pd.ExcelFile('split_csv/Stacking_indices39.xlsx')

# 加载 'Test Data' 数据表
df_test_data = pd.read_excel(xls, 'Test Data')

# 添加到 'SHAP Values' 数据表
shap_values_df = pd.concat([shap_values_df, df_test_data], axis=1)

# 删除 'Correct Prediction' 为 False 的行
shap_values_df = shap_values_df[shap_values_df['Correct Prediction'] != False]

# 删除 'Predicted Class' 为 1 的行
shap_values_df = shap_values_df[shap_values_df['Predicted Class'] != 0]

# 将更新后的数据表存储到一个新的 Excel 文件中
with pd.ExcelWriter('./table/output_values_filtered-被正确预测为患者的样本.xlsx') as writer:
    shap_values_df.to_excel(writer, sheet_name='SHAP Values Filtered')
    original_values_df.to_excel(writer, sheet_name='Original Feature Values')

### 决策路径（原始基于的SHAP values）

##### 一个样本

In [ ]:

index_sample = 5

# Get the feature values and SHAP values of the sample
T_single = X_test.loc[[index_sample]]
sh_single = shap_values[[index_sample]]

# Create a new figure with a specific size
plt.figure(figsize=(10, 8))

# Create the decision plot for the single sample
shap.decision_plot(explainer.expected_value, sh_single, T_single, feature_order='hclust', show=False)

# Adjust the subplot layout
plt.subplots_adjust(left=0.15, right=0.95, top=0.95, bottom=0.1)

# Save the figure
plt.savefig(f'image/decision_plot_single_sample-5.jpg', dpi=800, bbox_inches='tight')

# Show the plot
plt.show()

In [ ]:

index_sample = 33

# Get the feature values and SHAP values of the sample
T_single = X_test.loc[[index_sample]]
sh_single = sv[[index_sample]]

# Create a new figure with a specific size
plt.figure(figsize=(10, 8))

# Create the decision plot for the single sample
shap.decision_plot(explainer.expected_value, sh_single, T_single, feature_order='hclust', show=False)

# Adjust the subplot layout
plt.subplots_adjust(left=0.15, right=0.95, top=0.95, bottom=0.1)

# Save the figure
plt.savefig(f'image/decision_plot_single_sample-33.jpg', dpi=33, bbox_inches='tight')

# Show the plot
plt.show()

##### 所有被正确预测为患者和健康个体的的决策路径

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

# Set the font family to Arial
mpl.rcParams['font.family'] = 'Arial'

mpl.rcParams['font.weight'] = 'bold'

# Predict the class labels
y_pred = clf_stacking.predict(X_test)

# Get the indices of the samples correctly predicted as 0
correct_0_indices = X_test[(y_test == 1) & (y_pred == 1)].index

# Get the indices of the samples correctly predicted as 1
correct_1_indices = X_test[(y_test == 0) & (y_pred == 0)].index

# Combine the correct indices
correct_indices = correct_0_indices.append(correct_1_indices)

# Get the feature values and SHAP values of the samples
T = X_test.loc[correct_indices]
sh = sv[correct_indices]

plt.figure(figsize=(10, 8))
shap.decision_plot(explainer.expected_value, sh, T, feature_order='hclust', show=False)

plt.subplots_adjust(left=0.40, right=0.95, top=0.70, bottom=0.1)

plt.title('Individual-level decision path', fontsize=22, pad=50, fontweight='bold', x=0.5)
plt.xlabel('Model output value', fontsize=16)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.legend(fontsize=12)

plt.tick_params(axis='x', which='major', labelsize=12)
plt.tick_params(axis='y', which='major', labelsize=14)  # Adjust the font size for y-axis labels


plt.grid(True, linewidth=0.55, color='gray', alpha=0.75, axis='y')

plt.subplots_adjust(left=0.15, right=0.95, top=0.95, bottom=0.1)
plt.savefig(f'image/decision_plot_correct所有被正确预测为患者和健康个体的的决策路径.jpg', dpi=800, bbox_inches='tight')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

# Set the font family to Arial
mpl.rcParams['font.family'] = 'Arial'

mpl.rcParams['font.weight'] = 'bold'

# Predict the class labels
y_pred = clf_stacking.predict(X_test)

# Get the indices of the samples correctly predicted as 0
correct_0_indices = X_test[(y_test == 1) & (y_pred == 1)].index

# Get the indices of the samples correctly predicted as 1
correct_1_indices = X_test[(y_test == 0) & (y_pred == 0)].index

# Combine the correct indices
correct_indices = correct_0_indices.append(correct_1_indices)

# Get the feature values and SHAP values of the samples
T = X_test.loc[correct_indices]
sh = sv[correct_indices]

plt.figure(figsize=(10, 8))
shap.decision_plot(explainer.expected_value, sh, T, feature_order='hclust', show=False)

plt.subplots_adjust(left=0.40, right=0.95, top=0.70, bottom=0.1)

plt.xlabel('Model output value', fontsize=16)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.legend(fontsize=12)

plt.tick_params(axis='x', which='major', labelsize=16)
plt.tick_params(axis='y', which='major', labelsize=18)  # Adjust the font size for y-axis labels


plt.grid(True, linewidth=0.55, color='gray', alpha=0.75, axis='y')

plt.subplots_adjust(left=0.15, right=0.95, top=0.95, bottom=0.1)
plt.savefig(f'image/decision_plot_correct所有被正确预测为患者和健康个体的的决策路径x.jpg', dpi=500, bbox_inches='tight')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

# Set the font family to Arial
mpl.rcParams['font.family'] = 'Arial'

mpl.rcParams['font.weight'] = 'bold'

# Predict the class labels
y_pred = clf_stacking.predict(X_test)

# Get the indices of one sample correctly predicted as 0 and one sample correctly predicted as 1
correct_0_index = X_test[(y_test == 1) & (y_pred == 1)].index[5]
correct_1_index = X_test[(y_test == 1) & (y_pred == 1)].index[8]
correct_2_index = X_test[(y_test == 1) & (y_pred == 1)].index[15]
# Combine the selected indices
selected_indices = [correct_0_index, correct_1_index, correct_2_index]

# Get the feature values and SHAP values of the selected samples
T_selected = X_test.loc[selected_indices]
sh_selected = sv[selected_indices]

plt.figure(figsize=(10, 8))
shap.decision_plot(explainer.expected_value, sh_selected, T_selected, feature_order='hclust', show=False)

# Get the y-tick labels
ylabels = plt.gca().get_yticklabels()

# Only keep the labels at odd indices, set the rest to empty strings
for i in range(len(ylabels)):
    if i % 2 == 0:  # If the index is even
        ylabels[i] = ''

# Set the modified y-tick labels
plt.gca().set_yticklabels(ylabels)

plt.subplots_adjust(left=0.90, right=0.95, top=5.70, bottom=0.1)

plt.xlabel('Model output value', fontsize=20)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.legend(fontsize=12)

plt.tick_params(axis='x', which='major', labelsize=16)
plt.tick_params(axis='y', which='major', labelsize=20)  # Adjust the font size for y-axis labels

plt.grid(True, linewidth=0.55, color='gray', alpha=0.75, axis='y')

plt.subplots_adjust(left=0.15, right=0.95, top=0.95, bottom=0.1)
plt.savefig(f'image/decision_plot_correct所有被正确预测为患者和健康个体的的决策路径x.jpg', dpi=790, bbox_inches='tight')
plt.show()

##### 所有被正确预测为患者的决策路径

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

# Set the font family to Arial
mpl.rcParams['font.family'] = 'Arial'

mpl.rcParams['font.weight'] = 'bold'

# Predict the class labels
y_pred = clf_stacking.predict(X_test)

# Get the indices of the samples correctly predicted as 0
correct_0_indices = X_test[(y_test == 1) & (y_pred == 1)].index

# Get the feature values and SHAP values of the samples
T = X_test.loc[correct_0_indices]
sh = sv[correct_0_indices]

# Create a new figure with a specific size
plt.figure(figsize=(10, 8))

# Create the decision plot
shap.decision_plot(explainer.expected_value, sh, T, feature_order='hclust', show=False)

# Adjust the subplot layout
plt.subplots_adjust(left=0.40, right=0.95, top=0.70, bottom=0.1)

# Increase font size and line width
plt.title('Individuals correctly predicted as schizophrenia ', fontsize=16, pad=40, fontweight='bold', x=0.5)
plt.xlabel('Model output value', fontsize=16)
plt.tick_params(axis='x', which='major', labelsize=14)
plt.tick_params(axis='y', which='major', labelsize=10)  # Adjust the font size for y-axis labels
plt.legend(fontsize=14)


plt.grid(True, linewidth=0.43, color='gray', alpha=0.75, axis='y')  

# Save the figure
plt.savefig(f'image/decision_plot_correct_所有被正确预测为患者.jpg', dpi=1200, bbox_inches='tight')

# Show the plot
plt.show()

##### 抑制能力不损伤的，但是被正确预测为患者的

In [ ]:

# 预测类别标签
y_pred = clf_stacking.predict(X_test)

# 获取被正确预测为1的样本的索引
correct_1_indices = X_test[(y_test == 1) & (y_pred == 1)].index

# 获取样本的特征值和SHAP值
T = X_test.loc[correct_1_indices]  
sh = sv[correct_1_indices]

# 创建一个布尔数组,指示'Stroop_incongruent_rt', 'Stroop_congruent_rt', 'Stroop_neutral_rt' 特征是否同时为负值
is_highlight = (T['Stroop_incongruent_rt'] < 0) & (T['Stroop_congruent_rt'] < 0) & (T['Stroop_neutral_rt'] < 0)

# 根据is_highlight筛选出满足条件的样本
T_filtered = T[is_highlight]
sh_filtered = sh[is_highlight]

# 创建一个具有特定尺寸的新图形
plt.figure(figsize=(10, 8))

# 使用筛选后的数据创建决策图
shap.decision_plot(explainer.expected_value, sh_filtered, T_filtered, highlight=is_highlight[is_highlight], feature_order='hclust', show=False)

# 调整子图布局
plt.subplots_adjust(left=0.40, right=0.95, top=0.70, bottom=0.1)

# 增加字体大小和线宽


plt.title('Individuals with unimpaired interference inhibition', fontsize=20, pad=40, fontweight='bold', x=0.3)
plt.xlabel('Model output value', fontsize=18)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.legend(fontsize=14)

# 获取当前坐标轴的所有线条对象
lines = plt.gca().lines

plt.grid(True, linewidth=0.33, color='gray', alpha=0.75, axis='y')  
# 保存图形
plt.savefig(f'image/decision_plot_correct_干扰抑制能力不损伤.jpg', dpi=1200, bbox_inches='tight')

# 显示图形
plt.show()

##### 刷新能力不损伤的，但是被正确预测为患者的

In [ ]:

# 预测类别标签
y_pred = clf_stacking.predict(X_test)

# 获取被正确预测为1的样本的索引
correct_1_indices = X_test[(y_test == 1) & (y_pred == 1)].index

# 获取样本的特征值和SHAP值
T = X_test.loc[correct_1_indices]  
sh = sv[correct_1_indices]

# 创建一个布尔数组,指示'Stroop_incongruent_rt', 'Stroop_congruent_rt', 'Stroop_neutral_rt' 特征是否同时为负值
is_highlight = (T['RM-1,750_acc'] > 0) & (T['RM-750_acc'] > 0)
# 根据is_highlight筛选出满足条件的样本
T_filtered = T[is_highlight]
sh_filtered = sh[is_highlight]

# 创建一个具有特定尺寸的新图形
plt.figure(figsize=(10, 8))

# 使用筛选后的数据创建决策图
shap.decision_plot(explainer.expected_value, sh_filtered, T_filtered, highlight=is_highlight[is_highlight], feature_order='hclust', show=False)

# 调整子图布局
plt.subplots_adjust(left=0.40, right=0.95, top=0.70, bottom=0.1)

# 增加字体大小和线宽


plt.title('Individuals with unimpaired working memory updating', fontsize=20, pad=40, fontweight='bold', x=0.3)
plt.xlabel('Model output value', fontsize=18)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.legend(fontsize=14)

# 获取当前坐标轴的所有线条对象
lines = plt.gca().lines

plt.grid(True, linewidth=0.33, color='gray', alpha=0.75, axis='y')  
# 保存图形
plt.savefig(f'image/decision_plot_correct_刷新能力不损伤.jpg', dpi=1200, bbox_inches='tight')

# 显示图形
plt.show()

##### 干扰抑制能力不损伤的，但是被正确预测为患者的

In [ ]:

# 预测类别标签
y_pred = clf_stacking.predict(X_test)

# 获取被正确预测为1的样本的索引
correct_1_indices = X_test[(y_test == 1) & (y_pred == 1)].index

# 获取样本的特征值和SHAP值
T = X_test.loc[correct_1_indices]  
sh = sv[correct_1_indices]

# 创建一个布尔数组,指示'Stroop_incongruent_rt', 'Stroop_congruent_rt', 'Stroop_neutral_rt' 特征是否同时为负值
is_highlight = (T['Nogo_acc'] > 0) & (T['Go_acc'] > 0) & (T['Go_rt'] < 0)
# 根据is_highlight筛选出满足条件的样本
T_filtered = T[is_highlight]
sh_filtered = sh[is_highlight]

# 创建一个具有特定尺寸的新图形
plt.figure(figsize=(10, 8))

# 使用筛选后的数据创建决策图
shap.decision_plot(explainer.expected_value, sh_filtered, T_filtered, highlight=is_highlight[is_highlight], feature_order='hclust', show=False)

# 调整子图布局
plt.subplots_adjust(left=0.40, right=0.95, top=0.70, bottom=0.1)

# 增加字体大小和线宽


plt.title('Individuals with unimpaired response inhibition', fontsize=20, pad=40, fontweight='bold', x=0.3)
plt.xlabel('Model output value', fontsize=18)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.legend(fontsize=14)

# 获取当前坐标轴的所有线条对象
lines = plt.gca().lines

plt.grid(True, linewidth=0.33, color='gray', alpha=0.75, axis='y')  
# 保存图形
plt.savefig(f'image/decision_plot_correct_反应抑制能力不损伤.jpg', dpi=1200, bbox_inches='tight')

# 显示图形
plt.show()

##### 转换能力不损伤的，但是被正确预测为患者的

In [ ]:

# 预测类别标签
y_pred = clf_stacking.predict(X_test)

# 获取被正确预测为1的样本的索引
correct_1_indices = X_test[(y_test == 1) & (y_pred == 1)].index

# 获取样本的特征值和SHAP值
T = X_test.loc[correct_1_indices]  
sh = sv[correct_1_indices]

# 创建一个布尔数组,指示'Stroop_incongruent_rt', 'Stroop_congruent_rt', 'Stroop_neutral_rt' 特征是否同时为负值
is_highlight = (T['Switch_cost'] < 0) & (T['Mixing_cost'] > 0)

# 根据is_highlight筛选出满足条件的样本
T_filtered = T[is_highlight]
sh_filtered = sh[is_highlight]

# 创建一个具有特定尺寸的新图形
plt.figure(figsize=(10, 8))

# 使用筛选后的数据创建决策图
shap.decision_plot(explainer.expected_value, sh_filtered, T_filtered, highlight=is_highlight[is_highlight], feature_order='hclust', show=False)

# 调整子图布局
plt.subplots_adjust(left=0.40, right=0.95, top=0.70, bottom=0.1)

# 增加字体大小和线宽


plt.title('Individuals with unimpaired Switching', fontsize=20, pad=40, fontweight='bold', x=0.3)
plt.xlabel('Model output value', fontsize=18)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.legend(fontsize=14)

# 获取当前坐标轴的所有线条对象
lines = plt.gca().lines

plt.grid(True, linewidth=0.33, color='gray', alpha=0.75, axis='y')  
# 保存图形
plt.savefig(f'image/decision_plot_correct_转换能力不损伤.jpg.jpg', dpi=1200, bbox_inches='tight')

# 显示图形
plt.show()

##### 工作记忆广度能力不损伤的，但是被正确预测为患者的

In [ ]:

# 预测类别标签
y_pred = clf_stacking.predict(X_test)

# 获取被正确预测为1的样本的索引
correct_1_indices = X_test[(y_test == 1) & (y_pred == 1)].index

# 获取样本的特征值和SHAP值
T = X_test.loc[correct_1_indices]  
sh = sv[correct_1_indices]

is_highlight =((T['DSBT_Span'] > 0) &
    (T['CBT_Span'] > 0))

# 根据is_highlight筛选出满足条件的样本
T_filtered = T[is_highlight]
sh_filtered = sh[is_highlight]

# 创建一个具有特定尺寸的新图形
plt.figure(figsize=(10, 8))

# 使用筛选后的数据创建决策图
shap.decision_plot(explainer.expected_value, sh_filtered, T_filtered, highlight=is_highlight[is_highlight], feature_order='hclust', show=False)

# 调整子图布局
plt.subplots_adjust(left=0.40, right=0.95, top=0.70, bottom=0.1)

# 增加字体大小和线宽


plt.title('Individuals with unimpaired Working memory Maintenance', fontsize=20, pad=40, fontweight='bold', x=0.3)
plt.xlabel('Model output value', fontsize=18)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.legend(fontsize=14)

# 获取当前坐标轴的所有线条对象
lines = plt.gca().lines

plt.grid(True, linewidth=0.33, color='gray', alpha=0.75, axis='y')  
# 保存图形
plt.savefig(f'image/decision_plot_correct_工作记忆广度能力不损伤.jpg.jpg', dpi=1200, bbox_inches='tight')

# 显示图形
plt.show()

#### 执行功能都不损伤

In [ ]:
# Predict the class labels
y_pred = clf_stacking.predict(X_test)

# Get the indices of the samples correctly predicted as 0
correct_0_indices = X_test[(y_test == 1) & (y_pred == 1)].index

# Get the feature values and SHAP values of the samples
T = X_test.loc[correct_0_indices]
sh = sv[correct_0_indices]

# Create a boolean array indicating whether the 'Stroop_incongruent_rt' feature is negative

is_highlight = (
    (T['Stroop_incongruent_rt'] < 0) &
    (T['RM-1,750_acc'] > 0) &
    (T['Nogo_acc'] > 0) &
    (T['CBT_Span'] > 0) &
    (T['Switch_cost'] < 0) &
    (T['DSBT_Span'] > 0)
)

# Create a new figure with a specific size
plt.figure(figsize=(10, 8))


shap.decision_plot(explainer.expected_value, sh, T, highlight=is_highlight, feature_order='hclust', show=False)

# Adjust the subplot layout
plt.subplots_adjust(left=0.40, right=0.95, top=0.70, bottom=0.1)

# Increase font size and line width
plt.title('Individuals with no impairments on the three EF dimensions (none)', fontsize=20, pad=40, fontweight='bold', x=0.3)
plt.xlabel('Model output value', fontsize=18)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.legend(fontsize=14)

plt.grid(True, linewidth=0.33, color='gray', alpha=0.75, axis='y')  

# Save the figure
plt.savefig(f'image/decision_plot_correct_执行功能都不损伤.jpg', dpi=1200, bbox_inches='tight')

# Show the plot
plt.show()

#### 所有被正确预测为患者的平均预测路径

In [ ]:
# Calculate the mean feature values and SHAP values
T_mean = T.mean(axis=0)
sh_mean = sh.mean(axis=0)

# Create a new figure with a specific size
plt.figure(figsize=(10, 8))

# Create the decision plot
shap.decision_plot(explainer.expected_value, sh_mean, T_mean, feature_order='hclust', show=False)

# Remove the "Model output value" label
plt.gca().set_xlabel('')

# Adjust the subplot layout
plt.subplots_adjust(left=0.10, right=0.68, top=0.90, bottom=0.12)

# Increase font size and line width
plt.title('Group-level decision path in schizophrenia', fontsize=20, pad=50, fontweight='bold', x=0.4)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.legend(fontsize=14)

# Remove the bottom x-tick labels
plt.tick_params(axis='x', which='major', labelsize=12, labelbottom=False)
plt.tick_params(axis='y', which='major', labelsize=14)  # Adjust the font size for y-axis labels
plt.grid(True, linewidth=0.45, color='gray', alpha=0.75, axis='y')  

# Save the figure
plt.savefig(f'image/所有被正确预测为患者的平均预测路径.jpg', dpi=1200, bbox_inches='tight')

# Show the plot
plt.show()

#### Stacked force plot

In [ ]:
shap.force_plot(explainer.expected_value, shap_values, X_test)

In [ ]:
shap.force_plot(explainer.expected_value, shap_values, X_test)
import shap

shap_values = explainer.shap_values(X_test)
shap.initjs()


force_plot = shap.force_plot(explainer.expected_value, shap_values, X_test)

shap.save_html('data/force_plot.html', force_plot)

### Group level（整体加权）

#### Violin plot

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import shap

# Set the font family to Arial
mpl.rcParams['font.family'] = 'Arial'

mpl.rcParams['font.weight'] = 'bold'

# 设置全局的字体大小和粗细
mpl.rcParams['font.size'] = 16

# 计算每个特征的 SHAP 值的绝对值的平均值
mean_abs_shap_values = np.mean(np.abs(shap_values), axis=0)

# 生成 SHAP 值的小提琴图
shap.plots.violin(shap_values, X_test, plot_type="layered_violin", plot_size=(6.5, 9), show=False, max_display=12)

ax = plt.gca()

# 获取 y 轴标签
yticklabels = ax.get_yticklabels()

feature_ordering = np.argsort(np.sum(np.abs(shap_values), axis=0))[::-1]

# 设置 y 轴标签,并添加 SHAP 值
ax.set_yticklabels([f'{X_test.columns[feature_ordering[len(yticklabels)-i-1]]} ({mean_abs_shap_values[feature_ordering[len(yticklabels)-i-1]]:.4f})' for i in range(len(yticklabels))])

# 设置 x 轴和 y 轴的字体大小和粗细
plt.xticks(fontsize=15)
plt.yticks(fontsize=14, fontweight='bold')

plt.tick_params(axis='x', which='major', labelsize=12)
plt.tick_params(axis='y', which='major', labelsize=14) 

plt.title('Group-level feature importance', fontsize=20, fontweight='bold', pad=20)  # Adjust the title distance

# 设置 x 轴和 y 轴的标题,并指定字体大小
ax.set_xlabel('Mean SHAP Values', fontsize=18)
ax.set_ylabel('EF dimensions', fontsize=20)

# 保存图像
plt.savefig('image/summary_plot-小提琴图-蓝橙配色.jpg', dpi=1200, bbox_inches='tight')

# 显示图像
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import shap

# Set the font family to Arial
mpl.rcParams['font.family'] = 'Arial'

mpl.rcParams['font.weight'] = 'bold'

# 设置全局的字体大小和粗细
mpl.rcParams['font.size'] = 16

# 计算每个特征的 SHAP 值的绝对值的平均值
mean_abs_shap_values = np.mean(np.abs(shap_values), axis=0)

# 生成 SHAP 值的小提琴图
shap.plots.violin(shap_values, X_test, plot_type="layered_violin", plot_size=(6.5, 9), show=False, max_display=8)

ax = plt.gca()

# 获取 y 轴标签
yticklabels = ax.get_yticklabels()

feature_ordering = np.argsort(np.sum(np.abs(shap_values), axis=0))[::-1]

# 设置 y 轴标签,并添加 SHAP 值
ax.set_yticklabels([f'{X_test.columns[feature_ordering[len(yticklabels)-i-1]]} ({mean_abs_shap_values[feature_ordering[len(yticklabels)-i-1]]:.4f})' for i in range(len(yticklabels))])

# 设置 x 轴和 y 轴的字体大小和粗细
plt.xticks(fontsize=15)
plt.yticks(fontsize=14, fontweight='bold')

plt.tick_params(axis='x', which='major', labelsize=18)
plt.tick_params(axis='y', which='major', labelsize=18) 



# 设置 x 轴和 y 轴的标题,并指定字体大小
ax.set_xlabel('Mean SHAP Values', fontsize=20)
ax.set_ylabel('EF dimensions', fontsize=20)

# 保存图像
plt.savefig('image/summary_plot-小提琴图-蓝橙配色-8个.jpg', dpi=900, bbox_inches='tight')

# 显示图像
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import shap

# 计算每个特征的 SHAP 值的绝对值的平均值
mean_abs_shap_values = np.mean(np.abs(shap_values), axis=0)

# 设置全局的字体大小和粗细
mpl.rcParams['font.size'] = 14

# 生成 SHAP 值的小提琴图
shap.plots.violin(shap_values,X_test,plot_size=(10,12),show=False,max_display=35)

ax = plt.gca()

# 获取 y 轴标签
yticklabels = ax.get_yticklabels()

feature_ordering = np.argsort(np.sum(np.abs(shap_values), axis=0))[::-1]

# 设置 y 轴标签，并添加 SHAP 值
ax.set_yticklabels([f'{X_test.columns[feature_ordering[len(yticklabels)-i-1]]} ({mean_abs_shap_values[feature_ordering[len(yticklabels)-i-1]]:.4f})' for i in range(len(yticklabels))])

# 设置 x 轴和 y 轴的字体大小和粗细
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# 设置 x 轴和 y 轴的标题，并指定字体大小
ax.set_xlabel('Mean SHAP Values', fontsize=16)
ax.set_ylabel('EF dimensions', fontsize=16)

# 保存图像
plt.savefig('image/summary_plot-小提琴图-原始配色.jpg', dpi=800, bbox_inches='tight')

# 显示图像
plt.show()

#### beewarm plot

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import shap

# 计算每个特征的 SHAP 值的绝对值的平均值
mean_abs_shap_values = np.mean(np.abs(shap_values), axis=0)

# 设置全局的字体大小和粗细
mpl.rcParams['font.size'] = 14

# 生成 SHAP 值的小提琴图
shap.summary_plot(shap_values, X_test, plot_size=(10,12), show=False, max_display=35, cmap='viridis')


ax = plt.gca()

# 获取 y 轴标签
yticklabels = ax.get_yticklabels()

feature_ordering = np.argsort(np.sum(np.abs(shap_values), axis=0))[::-1]

# 设置 y 轴标签，并添加 SHAP 值
ax.set_yticklabels([f'{X_test.columns[feature_ordering[len(yticklabels)-i-1]]} ({mean_abs_shap_values[feature_ordering[len(yticklabels)-i-1]]:.4f})' for i in range(len(yticklabels))])

# 设置 x 轴和 y 轴的字体大小和粗细
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# 设置 x 轴和 y 轴的标题，并指定字体大小
ax.set_xlabel('Mean SHAP Values', fontsize=16)
ax.set_ylabel('EF dimensions', fontsize=16)

plt.savefig('image/summary_plot-散点图-绿黄配色.jpg', dpi=800, bbox_inches='tight')

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import shap

# 计算每个特征的 SHAP 值的绝对值的平均值
mean_abs_shap_values = np.mean(np.abs(shap_values), axis=0)

# 设置全局的字体大小和粗细
mpl.rcParams['font.size'] = 14

# 生成 SHAP 值的小提琴图
shap.summary_plot(shap_values, X_test, plot_size=(10,12), show=False, max_display=35)


ax = plt.gca()

# 获取 y 轴标签
yticklabels = ax.get_yticklabels()

feature_ordering = np.argsort(np.sum(np.abs(shap_values), axis=0))[::-1]

# 设置 y 轴标签，并添加 SHAP 值
ax.set_yticklabels([f'{X_test.columns[feature_ordering[len(yticklabels)-i-1]]} ({mean_abs_shap_values[feature_ordering[len(yticklabels)-i-1]]:.4f})' for i in range(len(yticklabels))])

# 设置 x 轴和 y 轴的字体大小和粗细
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# 设置 x 轴和 y 轴的标题，并指定字体大小
ax.set_xlabel('Mean SHAP Values', fontsize=16)
ax.set_ylabel('EF dimensions', fontsize=16)

plt.savefig('image/summary_plot-散点图-原始配色.jpg', dpi=800, bbox_inches='tight')

plt.show()

### 全部完成！

In [ ]:
print("全部完成!")